In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
%matplotlib inline

In [2]:
df = pd.read_csv('smsspamcollection.tsv',sep='\t')

In [4]:
df.head()

,label,message,length,punct
0,ham,"Go until jurong point, crazy.. Available only ...",111,9
1,ham,Ok lar... Joking wif u oni...,29,6
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,6
3,ham,U dun say so early hor... U c already then say...,49,6
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,2


In [5]:
df.isnull().sum()

label      0
message    0
length     0
punct      0
dtype: int64

In [6]:
df['label'].value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [7]:
from sklearn.model_selection import train_test_split

In [9]:
X = df['message']
y = df['label']

In [11]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,
                                                 random_state=42)

In [12]:
def predicting_mode(model):
    model.fit(X_train,y_train)
    y_preds = model.predict(X_test)
    
    y_score = model.socre(X_test,y_test)
    
    print(f"Scoring is {y_score*100}%")

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

In [14]:
count_vect = CountVectorizer()

In [16]:
# Fit vectorizer to the data (build a vocab, count number of word..)
# count_vect.fit(X_train)

# X_train_counts = count_vect.transform(X_train)

# Transform the original text --> Vector 

X_train_counts = count_vect.fit_transform(X_train)

In [17]:
X_train_counts

<3900x7263 sparse matrix of type '<class 'numpy.int64'>'
	with 52150 stored elements in Compressed Sparse Row format>

In [18]:
X_train_counts.shape

(3900, 7263)

In [19]:
from sklearn.feature_extraction.text import TfidfTransformer

In [20]:
tfidf_transformer = TfidfTransformer()

In [21]:
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [22]:
X_train_tfidf.shape

(3900, 7263)

In [23]:
X_train_tfidf

<3900x7263 sparse matrix of type '<class 'numpy.float64'>'
	with 52150 stored elements in Compressed Sparse Row format>

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [25]:
vector = TfidfVectorizer()

In [26]:
X_train_tfidf = vector.fit_transform(X_train)

In [27]:
from sklearn.svm import LinearSVC

In [28]:
clf = LinearSVC()

In [29]:
clf.fit(X_train_tfidf,y_train)

LinearSVC()

In [30]:
from sklearn.pipeline import Pipeline

In [31]:
text_clf = Pipeline([('tfidf',TfidfVectorizer()),('clf',LinearSVC())])

In [35]:
text_clf.fit(X_train,y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [36]:
text_clf.score(X_test,y_test)

0.9922248803827751

In [38]:
y_preds = text_clf.predict(X_test)

In [44]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

In [40]:
cm = confusion_matrix(y_test,y_preds)

In [41]:
cm

array([[1445,    3],
       [  10,  214]], dtype=int64)

In [43]:
print(classification_report(y_test,y_preds))

              precision    recall  f1-score   support

         ham       0.99      1.00      1.00      1448
        spam       0.99      0.96      0.97       224

    accuracy                           0.99      1672
   macro avg       0.99      0.98      0.98      1672
weighted avg       0.99      0.99      0.99      1672



In [46]:
print(f"accuracy score is {accuracy_score(y_test,y_preds)*100}%")

accuracy score is 99.22248803827752%


In [48]:
text_clf.predict(['Congratulations you have been selected as a winner please enter your details'])

array(['spam'], dtype=object)